In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup

In [2]:
!pip install moviepy


In [3]:
%cd /content
!git clone https://github.com/soCzech/TransNetV2.git

# !pip install tensorflow==2.1
!pip install tensorflow
!apt-get install ffmpeg
!pip install ffmpeg-python pillow


/content
Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 362 (delta 4), reused 4 (delta 4), pack-reused 355
Receiving objects: 100% (362/362), 98.15 KiB | 817.00 KiB/s, done.
Resolving deltas: 100% (210/210), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


## Test code cut clip from video

In [ ]:

from transnetv2 import TransNetV2

# location of learned weights is automatically inferred
# add argument model_dir="/path/to/transnetv2-weights/" to TransNetV2() if it fails
model_dir = "/content/TransNetV2/inference/transnetv2-weights"
model = TransNetV2(model_dir)
path_video = "/content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/L03_V001.mp4"
video_frames, single_frame_predictions, all_frame_predictions = \
    model.predict_video(path_video)

# # or
# video_frames = ... # np.array, shape: [n_frames, 27, 48, 3], dtype: np.uint8, RGB (not BGR)
# single_frame_predictions, all_frame_predictions = \
#     model.predict_frames(video_frames)

/content/TransNetV2/inference
[TransNetV2] Extracting frames from /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/L03_V001.mp4
[TransNetV2] Processing video frames 32910/32910


In [ ]:
list_of_scenes = model.predictions_to_scenes(predictions = single_frame_predictions)

In [ ]:
print("Total scenes in video = ", len(list_of_scenes))
print(type(list_of_scenes))
# print(list_of_scenes)
# print(list_of_scenes[1])

391
<class 'numpy.ndarray'>


In [ ]:
import matplotlib.pyplot as plt

# visualize scenes in video
ls = []
for i in range(len(list_of_scenes)):
  result = list_of_scenes[i]
  start_value = result[0]
  end_value = result[1]
  val = end_value - start_value
  ls.append(val)

# print(ls)
# print(type(ls))
print("max list of scenes = ", max(ls))
print("index max =", ls.index(max(ls)))
print("min list of scenes = ",min(ls))
print("index min = ", ls.index(min(ls)))
print("mean list of scenes = ", sum(ls)/len(ls))
plt.plot(ls)
plt.show()

In [ ]:
%cd /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

#function cut clip from video 
def cut_videos(input_video_path, output_video_path, list_of_scenes):
    video_clip = VideoFileClip(input_video_path)

    for start_frame, end_frame in list_of_scenes:
        start_time = start_frame/25
        end_time =  end_frame/25
        subclip = video_clip.subclip(start_time, end_time)

        subclip.write_videofile(output_video_path.format(start_frame, end_frame), codec="libx264")

    video_clip.reader.close()
    video_clip.audio.reader.close_proc()


input_video_path = "L03_V001.mp4"
output_video_path_template = "video_L03_V001_from_{}_to_{}.mp4"

cut_videos(input_video_path, output_video_path_template, list_of_scenes)


In [ ]:
%cd /content/TransNetV2

pil_image = model.visualize_predictions(
            frames = video_frames[19:500],
            predictions = (single_frame_predictions, all_frame_predictions))

display(pil_image)

--------------------------


## Code cut clip from video

**Formula to calculate the total frames in 1 video**

Mean frames in 1 scenes in 1 video = 25 FPS

```
total_frames = time_in_seconds * 25

Where:
    time_in_seconds: The duration of the video in seconds.
    25: Frames per second (FPS).
```

In [4]:
%cd /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video
!mkdir video_scenes_L03


/content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video


In [ ]:
import cv2
import glob
import os
%cd /content/TransNetV2/inference
from transnetv2 import TransNetV2

from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip


def cut_videos(input_video_path, output_video_path, list_of_scenes):
    video_clip = VideoFileClip(input_video_path)
    # Convert scenes to time ranges
    for start_frame, end_frame in scenes:
        start_time = start_frame/25
        end_time =  end_frame/25
        subclip = video_clip.subclip(start_time, end_time)

        subclip.write_videofile(output_video_path.format(start_frame, end_frame), codec="libx264")

    video_clip.reader.close()
    video_clip.audio.reader.close_proc()

model_dir = "/content/TransNetV2/inference/transnetv2-weights"
model = TransNetV2(model_dir)
video_paths = sorted(glob.glob('/content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Video_BTC/Video/video_L03/*.mp4'))
des_path = '/content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03'
for video_path in video_paths:
    folder_name = video_path.split('/')[-1].replace('.mp4', '')

    folder_path = os.path.join(des_path, folder_name)
    os.mkdir(folder_path)

    # TransNetV2
    video_frames, single_frame_predictions, all_frame_predictions = \
        model.predict_video(video_path)
    scenes = model.predictions_to_scenes(single_frame_predictions)

    # Cut and save videos based on scenes
    cut_videos(video_path, os.path.join(folder_path, "video_from_{}_to_{}.mp4"), scenes)


/content/TransNetV2/inference
[TransNetV2] Extracting frames from /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Video_BTC/Video/video_L03/L03_V001.mp4
[TransNetV2] Processing video frames 32910/32910
Moviepy - Building video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_0_to_19.mp4.
MoviePy - Writing audio in video_from_0_to_19TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_0_to_19.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_0_to_19.mp4
Moviepy - Building video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_20_to_455.mp4.
MoviePy - Writing audio in video_from_20_to_455TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_20_to_455.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_20_to_455.mp4
Moviepy - Building video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V001/video_from_456_to_507.mp4.
MoviePy - Writing audio in video_from_456_to_507TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2275_to_2482.mp4
Moviepy - Building video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2483_to_2546.mp4.
MoviePy - Writing audio in video_from_2483_to_2546TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2483_to_2546.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2483_to_2546.mp4
Moviepy - Building video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2547_to_2605.mp4.
MoviePy - Writing audio in video_from_2547_to_2605TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/0._HCM_AI_Challenge_2023/Data_Process/Cut_Video/video_scenes_L03/L03_V002/video_from_2547_to_2605.mp4



t:  97%|█████████▋| 57/59 [00:01<00:00, 28.89it/s, now=None]